In [1]:
import numpy as np
import time
import scipy.linalg as la

Následující kód implementuje Gaussovu eliminaci. Předně bych chtěl zdůraznit, že kód není napsán optimálně (pro rychlost), ale co nejjednodušeji (pro přehlednost), aby se v něm člověk snadno vyznal a mohl si ho osahat. Speciálně si můžete všimnout, že se v každém kole zbytečně odčítá a násobí celý řádek, přitom by stačily pouze prvky $j>i$. Pro praktické použití pochopitelně nemá smysl cokoliv programovat sám, můžeme místo toho využít nějakou vestavěnou knihovnu, která bude jistě pracovat daleko lépe.

Funkce `Gauss(A,b)` implementuje samotnou Gaussovu eliminaci. Přitom `A` je matice soustavy a `b` je vektor pravých stran (může být i matice s několika vektory ve sloupečcích). Soustava $Ax=b$ se převede na soustavu $Ux=d$, kde $U$ je horní trojúhelníková matice a $d$ nějaký jiný vektor. Výstupem je právě dvojice `[U,d]`.

Chceme-li získat skutečně řešení soustavy $Ax=b$, musíme po převodu na tvar $Ux=d$ provést tzv. zpětnou substituci. Tu provádí funkce `BS(U,d)`. Výstupem je řešení `x`.

**Cvičení:** Zkuste modifikovat funkci `Gauss(A,b)` tak, aby vygenerovala LU rozklad (když se vykašlete na pivotování), příp. LUP rozklad.

In [2]:
def Gauss(A,b):                      #GAUSSOVA ELIMINACE
    n=len(A)                         #n=velikost matice A
    B=np.c_[A,b]                     #B=rozšířená matice soustavy
    for i in range(n-1):             #procházíme všechny sloupce
        j=np.argmax(np.abs(B[i:,i])) #vybereme nejvyšší hodnotu v i-tém sloupci (počínaje i-tým řádkem)
        B[[i,i+j]]=B[[i+j,i]]        #prohodíme daný řádek s i-tým (částečné pivotování)
        for j in range(i+1,n):       #pro každé j>i vytvoříme nulu na j-tém řádku
            B[j]=B[j]-B[j,i]/B[i,i]*B[i]
    U=B[:,:n]                        #rozšířenou matici zpátky rozpůlíme na U a d
    d=B[:,n:]
    return(U,d)

In [3]:
def BS(U,d):                         #ZPĚTNÁ SUBSTITUCE
    n=len(U)                         #n=velikost matice U
    x=np.ndarray.copy(d)             #zkopírujeme zadané d, abychom si ho neponičili
    for i in range(n-1,-1,-1):       #procházíme odzadu řádky vektoru d
        x[i]=x[i]/U[i,i]
        for j in range(i):
            x[j]=x[j]-U[j,i]*x[i]
    return(x)

In [4]:
def GaussJordan(A,b):                #GAUSSOVA-JORDANOVA ELIMINACE
    n=len(A)                         #n=velikost matice A
    B=np.c_[A,b]                     #B=rozšířená matice soustavy
    for i in range(n):               #procházíme všechny sloupce
        j=np.argmax(np.abs(B[i:,i])) #vybereme nejvyšší hodnotu v i-tém sloupci (počínaje i-tým řádkem)
        B[[i,i+j]]=B[[i+j,i]]        #prohodíme daný řádek s i-tým (částečné pivotování)
        B[i]=B[i]/B[i,i]             #normalizujeme řádek
        for j in range(i):           #pro každé j!=i vytvoříme nulu na j-tém řádku
            B[j]=B[j]-B[j,i]*B[i]
        for j in range(i+1,n):         
            B[j]=B[j]-B[j,i]*B[i]
    x=B[:,n:]
    return(x)

Vyzkoušíme, jak to funguje, na příkladu ze cvičení

In [5]:
A=np.array([[2.,4.,-2.],[1.,1.,-2.],[-3.,1.,8.]])  #matice soustavy
b=np.array([4.,3.,-11.])                           #vektor/matice pravých stran

In [6]:
[U,d]=Gauss(A,b)

In [7]:
BS(U,d)

array([[ 1.],
       [ 0.],
       [-1.]])

In [8]:
GaussJordan(A,b)

array([[ 1.],
       [ 0.],
       [-1.]])

Následující kód generuje matice 1000×1000 a k nim vektory pravých stran a zkouší na ně různé metody a měří čas, za které to zvládnou. Kromě funkcí, které jsme implementovali výše, zkoušíme i vestavěnou funkci pro LUP rozklad (tj. Gaussova eliminace) a vestavěnou funkci pro řešení trojúhelníkových soustav (tj. zpětná substituce).

In [9]:
n=1000
A=2*np.random.rand(n,n)-np.ones((n,n))
b=2*np.random.rand(n)-np.ones(n)
t0=time.process_time()
[U,d]=Gauss(A,b)
t1=time.process_time()
BS(U,d)
t2=time.process_time()
GaussJordan(U,d)
t3=time.process_time()
la.lu(A)
t4=time.process_time()
la.solve_triangular(U,d)
t5=time.process_time()
print("Gaussova eliminace:  ",t1-t0)
print("Zpětná substituce:   ",t2-t1)
print("Gauss--Jordan:       ",t3-t2)
print("Vestavěný LU rozklad:",t4-t3)
print("Vestavěná BS:        ",t5-t4)

Gaussova eliminace:   1.9082108839999998
Zpětná substituce:    1.0811605870000003
Gauss--Jordan:        4.064199029
Vestavěný LU rozklad: 0.11547448199999977
Vestavěná BS:         0.027718140999999363


V následujícím kódu vygenerujeme matici $A$ spolu s LU rozkladem. (Dejme tomu, že už jsme jednou Gaussovu eliminaci pro $A$ provedli.) Nyní vezmeme novou pravou stranu $b$ a chceme porovnat, jak dlouho by trvalo opět provést celou Gaussovu eliminaci, oproti tomu, kdybychom využili LU rozklad a provedli pouze dopřednou a zpětnou substituci.

Pro provedení dopředné substituce používáme funkce `BS`, do které vložíme zrcadlově převrácenou matici a vektor pravých stran a výsledek opět zrcadlově převrátíme.

In [10]:
n=100
A=2*np.random.rand(n,n)-np.ones((n,n))
P,L,U=la.lu(A)
A=np.matmul(L,U)
b=np.random.rand(n)
t0=time.process_time()
[V,d]=Gauss(A,b)
t1=time.process_time()
x=BS(V,d)
t2=time.process_time()
y=np.flip(BS(np.flip(L),np.flip(b)))
t3=time.process_time()
z=BS(U,y)
t4=time.process_time()
print("Gaussova eliminace:  ",t1-t0)
print("Zpětná substituce:   ",t2-t1)
print("Dopředná substituce: ",t3-t2)
print("Zpětná substituce:   ",t4-t3)

Gaussova eliminace:   0.19963903500000058
Zpětná substituce:    0.15708388099999837
Dopředná substituce:  0.04420122800000037
Zpětná substituce:    0.04261610600000054


V následujícím kódu modifikujeme funkci `Gauss` tak, aby nedělala pivotování.

Obecně to bez pivotování nejde, protože se shodou okolností může stát, že se na diagonále objeví nula. V takovém případě je prohození řádků nutnost. Pro jednoduchost kódu tento detail zanedbáme a budeme doufat, že tato situace nenastane. V případě, že používáme náhodné matice je pravděpodobnost takové situace velmi nízká.

Porovnáme velikost chyby používajíce kód s pivotováním a bez něj.

In [11]:
def GaussNoPivot(A,b):               #GAUSSOVA ELIMINACE BEZ PIVOTOVÁNÍ
    n=len(A)                         #n=velikost matice A
    B=np.c_[A,b]                     #B=rozšířená matice soustavy
    for i in range(n-1):             #procházíme všechny sloupce
        for j in range(i+1,n):       #pro každé j>i vytvoříme nulu na j-tém řádku
            B[j]=B[j]-B[j,i]/B[i,i]*B[i]
    U=B[:,:n]                        #rozšířenou matici zpátky rozpůlíme na U a d
    d=B[:,n:]
    return(U,d)

In [12]:
n=200
A=2*np.random.rand(n,n)-np.ones((n,n))
x0=2*np.random.rand(n)-np.ones(n)
b=np.dot(A,x0)
[U1,d1]=Gauss(A,b)
x1=BS(U1,d1)
E1=np.linalg.norm(x1[:,0]-x0)
[U2,d2]=GaussNoPivot(A,b)
x2=BS(U2,d2)
E2=np.linalg.norm(x2[:,0]-x0)
E2/E1

138.13733652136747

Následující kód generuje náhodné matice a počítá jejich číslo podmíněnosti (pomocí řádkové normy). Můžeme chvíli zkoušet generovat a poté si do proměnné `B` uložit nějakou matici s poměrně malým číslem podmíněnosti (třeba <500) a do `C` si zkopírujeme matici s nějakým vysokým číslem podmíněnosti (třeba >10000).

Následně pro každou matici vygenerujeme náhodně dvacet pravých stran a spočítáme chybu řešení. Chyby následně zprůměrujeme Poměr chyb by měl řádově odpovídat poměru čísel podmíněnosti.

In [19]:
n=40
A=2*np.random.rand(n,n)-np.ones((n,n))
np.linalg.norm(A,np.infty)*np.linalg.norm(np.linalg.inv(A),np.infty)

30799.665932225416

In [17]:
B=np.ndarray.copy(A)

In [20]:
C=np.ndarray.copy(A)

In [21]:
condB=np.linalg.norm(B,np.infty)*np.linalg.norm(np.linalg.inv(B),np.infty)
condC=np.linalg.norm(C,np.infty)*np.linalg.norm(np.linalg.inv(C),np.infty)
print("Číslo podmíněnosti pro B: ",condB)
print("Číslo podmíněnosti pro C: ",condC)
print("Podíl: ",condC/condB)

Číslo podmíněnosti pro B:  401.1520315715057
Číslo podmíněnosti pro C:  30799.665932225416
Podíl:  76.7780380210672


In [22]:
E=0
for i in range(20):
    x0=2*np.random.rand(n)-np.ones(n)
    b=np.dot(B,x0)
    [U,d]=Gauss(B,b)
    x=BS(U,d)
    E=E+np.linalg.norm(x[:,0]-x0)
E1=E/20
print("Průměrná chyba pro B: ",E1)
E=0
for i in range(20):
    x0=2*np.random.rand(n)-np.ones(n)
    b=np.dot(C,x0)
    [U,d]=Gauss(C,b)
    x=BS(U,d)
    E=E+np.linalg.norm(x[:,0]-x0)
E2=E/20
print("Průměrná chyba pro C: ",E2)
print("Podíl: ",E2/E1)

Průměrná chyba pro B:  1.1238940856924418e-14
Průměrná chyba pro C:  7.540624461035936e-13
Podíl:  67.09372846632684
